In [ ]:
import os
import cv2
import pickle
import matplotlib.image as mpimg

project_dir = os.path.dirname(os.getcwd())
test_folder_path = os.path.join(project_dir, 'test_images')
output_folder_path = os.path.join(project_dir, 'output_images')

pickle_path = os.path.join(project_dir, 'dist_pickle.p')
dist_pickle = pickle.load(open(pickle_path, "rb"))
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]

def undistort_image(image_name):
    img = mpimg.imread(image_name)
    return img, cv2.undistort(img, mtx, dist, None, mtx)

In [ ]:
import numpy as np

def get_source_destination_points(width, height):
    top_offset = 100
    bottom_offset = 10
    top_line_offset = 75
    bottom_line_offset = 115
    src = np.array([
            (bottom_line_offset,height-bottom_offset), (width/2 - top_line_offset, height/2+top_offset), 
            (width/2 + top_line_offset, height/2+top_offset), (width-bottom_line_offset,height-bottom_offset)], dtype=np.int32)

    offset = 100
    new_height = height*10
    dst = np.float32([[offset, new_height], [offset, 0], [width-offset, 0], [width-offset, new_height]])
    return src, dst

 
src, dst = get_source_destination_points(1280, 720)
print('Source', src)
print('Destination', dst)
    
def get_region_of_interest(img, show_roi=True):
    new_image = img.copy()
    height = new_image.shape[0]
    width = new_image.shape[1]
    if show_roi == True:
        cv2.polylines(new_image, [src], True, (255, 0, 0), thickness=2)
    
    M = cv2.getPerspectiveTransform(src.astype(np.float32), dst)
    warped = cv2.warpPerspective(new_image, M, (width, 10*height))
    
    return new_image, warped

In [ ]:
import os
import glob
import matplotlib.pyplot as plt
%matplotlib inline


image_pattern = os.path.join(test_folder_path, '*.jpg')
images = glob.glob(image_pattern)

def test_region_of_interest(img_name):
    image_path = os.path.join(test_folder_path, img_name)
    image, undistorted = undistort_image(image_path)
    
    roi_mage, warped_image = get_region_of_interest(undistorted)
    
    output_image_path = os.path.join(output_folder_path, 'undistorted_'+img_name)
    mpimg.imsave(output_image_path, undistorted)
    output_image_path1 = os.path.join(output_folder_path, 'roi_'+img_name)
    mpimg.imsave(output_image_path1, roi_mage)
    output_image_path2 = os.path.join(output_folder_path, 'warped_'+img_name)
    mpimg.imsave(output_image_path2, warped_image)

    
    f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(48, 9))
    f.tight_layout()
    ax1.imshow(image)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(undistorted)
    ax2.set_title('Undistorted Image', fontsize=50)
    ax3.imshow(roi_mage)
    ax3.set_title('Region of Interest', fontsize=50)
    
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    
    plt.figure();
    plt.imshow(warped_image)
    plt.title('Bird-view Image', fontsize=10)
    
    
for name in images:
    test_region_of_interest(os.path.basename(name))